In [1]:
!pip install kaggle
!mkdir ~/.kaggle


In [2]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download mateohervas/dcsass-dataset

100% 1.35G/1.35G [00:14<00:00, 65.8MB/s]
100% 1.35G/1.35G [00:14<00:00, 101MB/s] 


In [4]:
import zipfile

# Extract the zip file
with zipfile.ZipFile('/content/dcsass-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [6]:
import os
import shutil

DIR = '/content/dcsass dataset/DCSASS Dataset/'

# Use this to move all video clips from the subfolders into the main class folders for DCSASS dataset
for folder in os.listdir(DIR):
    folder_url = os.path.join(DIR, folder)
    if folder == 'Shoplifting':  # Check if the folder is 'Shoplifting'
        for item in os.listdir(folder_url):
            subfolder_url = os.path.join(folder_url, item)
            if os.path.isdir(subfolder_url):
                for clip in os.listdir(subfolder_url):
                    clip_url = os.path.join(subfolder_url, clip)
                    shutil.move(clip_url, folder_url)  # Move clips to the main 'shoplifting' folder
                # The empty folders will be deleted after clips are moved out of it
                os.rmdir(subfolder_url)


In [7]:
import os
import shutil
#/content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_0.mp4
import pandas as pd

# Create shoplifting class based on the Labels given in DCSASS dataset
DIR = '/content/dcsass dataset/DCSASS Dataset/Shoplifting/'
OUT = '/content/DCSASS Dataset/Shoplifting/'

if not os.path.isdir(OUT):
    os.mkdir(OUT)

label_file = '/content/dcsass dataset/DCSASS Dataset/Labels/Shoplifting.csv'
if os.path.isfile(label_file):
    label = pd.read_csv(label_file, header=None)
    for i in range(len(label)):
        if label[2][i] == 1:  # Check if the label indicates shoplifting (1)
            video_path = os.path.join(DIR, label[0][i] + '.mp4')
            if os.path.exists(video_path):
                dest = os.path.join(OUT, label[0][i] + '.mp4')
                if not os.path.exists(dest):
                    try:
                        shutil.move(video_path, OUT)
                        print(f"Moved {video_path} to {OUT}")
                    except Exception as e:
                        print(f"Error moving {video_path} to {OUT}: {e}")
                else:
                    print(f"File {dest} already exists in {OUT}, skipping move operation")
            else:
                print(f"Source file {video_path} does not exist, skipping move operation")
else:
    print(f"File not found: {label_file}")


Moved /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting001_x264_9.mp4 to /content/DCSASS Dataset/Shoplifting/
Moved /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting001_x264_10.mp4 to /content/DCSASS Dataset/Shoplifting/
Moved /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting001_x264_11.mp4 to /content/DCSASS Dataset/Shoplifting/
Moved /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting001_x264_12.mp4 to /content/DCSASS Dataset/Shoplifting/
Moved /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting001_x264_13.mp4 to /content/DCSASS Dataset/Shoplifting/
Moved /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting001_x264_14.mp4 to /content/DCSASS Dataset/Shoplifting/
Moved /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting001_x264_15.mp4 to /content/DCSASS Dataset/Shoplifting/
Moved /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting005_x264_12.mp4 to /content/DCSASS Dataset/Shoplifting/
Moved /co

In [8]:
import os
from os import listdir

main_folder = '/content/dcsass dataset/DCSASS Dataset/Shoplifting/'
fps = 25

for file in listdir(main_folder):
    file_url = os.path.join(main_folder, file)
    if os.path.isfile(file_url):
        if not os.path.isdir(file_url[:-4]):
            os.mkdir(file_url[:-4])
            print("Extracting frames from ", file_url)
            os.system('ffmpeg -i "{}" -vf fps={} "{}/%05d.jpg"'.format(file_url, fps, file_url[:-4]))
            os.remove(file_url)


Extracting frames from  /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting007_x264_13.mp4
Extracting frames from  /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting001_x264_21.mp4
Extracting frames from  /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting047_x264_6.mp4
Extracting frames from  /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting015_x264_6.mp4
Extracting frames from  /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting009_x264_23.mp4
Extracting frames from  /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting010_x264_6.mp4
Extracting frames from  /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting054_x264_4.mp4
Extracting frames from  /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting053_x264_15.mp4
Extracting frames from  /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting036_x264_9.mp4
Extracting frames from  /content/dcsass dataset/DCSASS Dataset/Shoplifting/Shoplifting0

In [9]:
# Import libraries
%matplotlib inline
import numpy as np
import os
import cv2
import matplotlib
import matplotlib.pyplot as plt
import math
import keras
from keras.layers import (Input, Activation, Conv3D, Dense, Dropout, Flatten, MaxPooling3D, BatchNormalization, AveragePooling3D, Reshape, Lambda, GlobalAveragePooling3D, Concatenate, ReLU, Add)
from keras.losses import categorical_crossentropy
from keras.models import Sequential, Model, load_model
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow as tf

In [10]:
def plot_history(history, result_dir):
    '''
    Plots the accuracy and loss graphs of train and val and saves them.
    '''

    plt.plot(history.history['accuracy'], marker='.')
    plt.plot(history.history['val_accuracy'], marker='.')
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['accuracy', 'val_accuracy'], loc='lower right')
    plt.savefig(os.path.join(result_dir, 'model_accuracy.png'))
    plt.show()

    plt.plot(history.history['loss'], marker='.')
    plt.plot(history.history['val_loss'], marker='.')
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.legend(['loss', 'val_loss'], loc='upper right')
    plt.savefig(os.path.join(result_dir, 'model_loss.png'))
    plt.show();

In [11]:
class Videoto3D:

    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth

    def video3d(self, filename):

        frames = []
        index = len(os.listdir(filename)) // self.depth
        images = os.listdir(filename)[::index]
        images = images[0:25]
        images.sort()

        for img in images:

            img_path = os.path.join(filename, img)
            frame = cv2.imread(img_path)
            frame = cv2.resize(frame, (self.height, self.width))
            frames.append(frame)

        return np.array(frames) / 255.0

In [12]:
from keras.utils import to_categorical
import os

def preprocess(video_dir, result_dir, nb_classes=2, img_size=224, frames=25):
    '''
    Preprocess the videos into X and Y and save in npz format and
    compute input shape
    '''

    img_rows, img_cols = img_size, img_size
    channel = 3

    target_class='Shoplifting'

    X = []
    labels = []
    labellist = []

    # Obtain labels and X
    name = os.path.join(video_dir, target_class)

    for v_files in os.listdir(name):

        v_file_path = os.path.join(name, v_files)
        label = target_class
        if label not in labellist:
            if len(labellist) >= nb_classes:
                continue
            labellist.append(label)
        labels.append(label)
        X.append(v_file_path)

    if not os.path.isdir(result_dir):
        os.mkdir(result_dir)
    with open(os.path.join(result_dir, 'classes.txt'), 'w') as fp:
        for i in range(len(labellist)):
            fp.write('{} {}\n'.format(i, labellist[i]))

    for num, label in enumerate(labellist):
        for i in range(len(labels)):
            if label == labels[i]:
                labels[i] = num

    Y = to_categorical(labels, nb_classes)

    print('X_shape:{}\tY_shape:{}'.format(len(X), Y.shape))

    input_shape = (frames, img_rows, img_cols, channel)

    return X, Y, input_shape



In [13]:
%run /content/models/slowfast.py

def resnet50(inputs, **kwargs):
    model = SlowFast_body(inputs, [3, 4, 6, 3], bottleneck, **kwargs)
    return model

In [14]:
class batchGenerator(keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size, vid3d):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.vid3d = vid3d

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = []

        for video in self.x[idx * self.batch_size:(idx + 1) * self.batch_size]:
            batch_x.append(self.vid3d.video3d(video))

        batch_x = np.array(batch_x)
        batch_x = batch_x.astype('float32')
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return batch_x, batch_y

In [15]:
def main(video_dir, result_dir, batch_size=32, epochs=100, img_size=224, frames=25):
    # Preprocess only shoplifting videos
    X, Y, input_shape = preprocess(video_dir, result_dir, nb_classes=2, img_size=img_size, frames=frames)

    # Print input shape
    print("Input Shape = ", input_shape)

    # Instantiate Videoto3D class
    vid3d = Videoto3D(width=img_size, height=img_size, depth=frames)

    # Define the model architecture (either SlowFast or I3D)
    x = Input(shape=input_shape)
    model = resnet50(x, num_classes=2)  # Adjust this based on your choice of architecture

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
                  metrics=['accuracy'])

    # Print model summary
    model.summary()

    # Split the dataset into training, validation, and testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, shuffle=True)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.15, shuffle=True)

    # Print dataset shapes
    print("X_train.shape = ", len(X_train))
    print("X_val.shape = ", len(X_val))
    print("X_test.shape = ", len(X_test))

    # Define callbacks for model checkpoints and logging
    csv_logger = CSVLogger(os.path.join(result_dir, 'shoplifting_model_history_log.csv'), append=True)
    filepath = os.path.join(result_dir, 'shoplifting-{epoch:02d}-{val_accuracy:.2f}.hd5')
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, mode='max')
    callbacks_list = [checkpoint, csv_logger]

    # Train the model
    history = model.fit(batchGenerator(X_train, Y_train, batch_size, vid3d),
                        steps_per_epoch=math.ceil(len(X_train) / batch_size),
                        validation_data=batchGenerator(X_val, Y_val, batch_size, vid3d),
                        validation_steps=math.ceil(len(X_val) / batch_size),
                        epochs=epochs, verbose=1, callbacks=callbacks_list, initial_epoch=0)

    # Save model architecture as JSON
    model_json = model.to_json()
    with open(os.path.join(result_dir, 'shoplifting.json'), 'w') as json_file:
        json_file.write(model_json)

    # Save model weights and final model
    model.save_weights(os.path.join(result_dir, 'shoplifting_finalweights.hd5'))
    model.save(os.path.join(result_dir, 'shoplifting_finalmodel.hd5'))

    # Evaluate the model on the testing set
    loss, acc = model.evaluate(batchGenerator(X_test, Y_test, batch_size, vid3d),
                               steps=math.ceil(len(X_test) / batch_size), verbose=1)

    # Plot training history
    plot_history(history, result_dir)

    # Print test loss and accuracy
    print('Test loss:', loss)
    print('Test accuracy:', acc)

    return history, model


In [16]:
## TRAIN MODEL ##

history, model = main(video_dir = '/content/dcsass dataset/DCSASS Dataset', result_dir = 'output/', batch_size = 8, epochs = 100, img_size = 224, frames = 25)

X_shape:741	Y_shape:(741, 2)
Input Shape =  (25, 224, 224, 3)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 25, 224, 224, 3)]    0         []                            
                                                                                                  
 data_fast (Lambda)          (None, 13, 224, 224, 3)      0         ['input_1[0][0]']             
                                                                                                  
 data_slow (Lambda)          (None, 2, 224, 224, 3)       0         ['input_1[0][0]']             
                                                                                                  
 sequential (Sequential)     (None, 13, 112, 112, 8)      5912      ['data_fast[0][0]']           
                                

KeyboardInterrupt: 

In [17]:
def frames_from_video(video_dir, nb_frames = 25, img_size = 224):

    # Opens the Video file
    cap = cv2.VideoCapture(video_dir)
    i=0
    frames = []
    while(cap.isOpened() and i<nb_frames):
        ret, frame = cap.read()
        if ret == False:
            break
        frame = cv2.resize(frame, (img_size, img_size))
        frames.append(frame)
        i+=1

    cap.release()
    cv2.destroyAllWindows()
    return np.array(frames) / 255.0

In [18]:
def predictions(video_dir, model, nb_frames = 25, img_size = 224):

    X = frames_from_video(video_dir, nb_frames, img_size)
    X = np.reshape(X, (1, nb_frames, img_size, img_size, 3))

    predictions = model.predict(X)
    preds = predictions.argmax(axis = 1)

    classes = []
    with open(os.path.join('output', 'classes.txt'), 'r') as fp:
        for line in fp:
            classes.append(line.split()[1])

    for i in range(len(preds)):
        print('Prediction - {} -- {}'.format(preds[i], classes[preds[i]]))

In [20]:
## LOAD MODEL ##

model = load_model('/content/output/shoplifting-56-1.00.hd5')

In [24]:
## MAKE PREDICTIONS ##

predictions(video_dir = '/content/test/Shoplifting018_x264_19.mp4', model = model, nb_frames = 25, img_size = 224)

1/1 [==============================] - 0s 39ms/step
Prediction - 0 -- Shoplifting
